# Multi-Agent Debate Demo

## Setup and Imports

In [1]:
# Add project root to path
import sys
sys.path.insert(0, '..')

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# Import debate components
from agents import (
    DebateOrchestrator,
    create_debate_orchestrator,
    DebateManager,
    DebateState,
    Recommendation,
    get_llm
)

# Import evaluation components
from evaluation import DebateEvaluator

# Import configuration
from utils.config import get_debate_config

print("Imports successful!")

c:\Users\Ryan Chin\Documents\UM CS\WIH3001 Data Science Project\RoundtableAI\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Imports successful!


## Initialize the LLM (Google Gemini)

In [ ]:
llm = get_llm(
    model_name="gemini-2.0-flash",
    temperature=0.3
)

Initializing Gemini model: gemini-2.0-flash
Successfully connected to Gemini model: gemini-2.0-flash


## 3. Create the Debate Orchestrator

In [4]:
# View debate configuration
debate_config = get_debate_config()
print("Debate Configuration:")
for key, value in debate_config.items():
    print(f"  {key}: {value}")

Debate Configuration:
  max_rounds: 5
  min_turns_per_agent: 2
  consensus_threshold: 0.75
  agent_order: ['fundamental', 'sentiment', 'valuation']
  synthesis_model: llama3.1:8b
  default_confidence: 0.5
  agent_weights: {'fundamental': 0.4, 'sentiment': 0.25, 'valuation': 0.35}


In [ ]:
# Callback to track debate progress
def on_message(message):
    agent = message.agent_type
    rec = message.recommendation.value
    conf = message.confidence
    round_num = message.round_number
    print(f"[Round {round_num}] {agent.upper()}: {rec} ({conf:.0%} confidence)")

def on_round_complete(round_result):
    round_num = round_result.round_number
    consensus = round_result.consensus_percentage
    leading = round_result.leading_recommendation.value
    print(f"\n--- Round {round_num} Complete ---")
    print(f"Consensus: {consensus:.0%}, Leading: {leading}")
    print()

In [6]:
# Create the orchestrator
orchestrator = create_debate_orchestrator(
    llm=llm,
    max_rounds=5,
    consensus_threshold=0.75,
    on_message_callback=on_message
)

print("Orchestrator created successfully!")
print(f"Agents: {list(orchestrator.agents.keys())}")

Orchestrator created successfully!
Agents: ['fundamental', 'sentiment', 'valuation']


## Query Classification

In [ ]:
# Multi-agent category
user_query = "Should I invest in Maybank? I'm looking for a balanced investment."

print("Processing query...")

result = orchestrator.route_query(user_query)

print(f"\nRoute Type: {result['route_type']}")
print(f"Risk Tolerance: {result.get('risk_tolerance', 'N/A')}")
print(f"Classification: {result.get('classification', {})}")

In [ ]:
# Display result based on route type
if result['route_type'] == 'debate':
    print("\n" + "="*60)
    print("MULTI-AGENT DEBATE RESULT")
    print("="*60)
    
    print(f"\nRecommendation: {result['recommendation']}")
    print(f"Confidence: {result['confidence']:.0%}")
    print(f"Consensus Level: {result['consensus']:.0%}")
    print(f"Risk Tolerance Applied: {result['risk_tolerance'].upper()}")
    
    # Access full result for more details
    final_rec = result['full_result']
    
    print("\nKey Points:")
    for point in final_rec.key_points:
        print(f"  - {point}")

    print("\nKey Risks:")
    for risk in final_rec.risks:
        print(f"  - {risk}")
        
elif result['route_type'] == 'single_agent':
    print("\n" + "="*60)
    print("SINGLE AGENT RESPONSE")
    print("="*60)
    print(f"\nAgent Used: {result['agent_used']}")
    print(f"Response:\n{result['response'][:1000]}...")
    
else:
    print(f"\nError: {result['response']}")

In [ ]:
# View agent breakdown
if result['route_type'] == 'debate':
    final_rec = result['full_result']
    print("Agent Breakdown:")
    print("-"*40)
    for agent_type, vote in final_rec.agent_breakdown.items():
        print(f"\n{agent_type.upper()}:")
        print(f"  Recommendation: {vote.recommendation.value}")
        print(f"  Confidence: {vote.confidence:.0%}")
        print(f"  Reasoning: {vote.reasoning[:200]}...")

## Risk Tolerance Inference

In [ ]:
# Test risk tolerance inference with different query styles

test_queries = [
    # Conservative queries
    "I'm nearing retirement and want a safe dividend stock. Is Maybank suitable for capital preservation?",
    
    # Moderate queries  
    "Looking for a balanced investment in CIMB with reasonable growth potential.",
    
    # Aggressive queries
    "I want maximum growth and don't mind volatility. Should I buy Public Bank for high returns?"
]

print("Risk Tolerance Inference Demo")
print("="*60)

for query in test_queries:
    print(f"\nQuery: {query[:80]}...")
    
    # Just classify without running full debate
    classification = orchestrator.classify_query(query)
    
    print(f"  → Risk Tolerance: {classification['risk_tolerance'].upper()}")
    print(f"  → Company: {classification.get('company', 'N/A')}")
    print(f"  → Needs Debate: {classification['needs_debate']}")
    print(f"  → Reasoning: {classification['reasoning'][:100]}...")

## Single Agent Queries

In [ ]:
# Example: Single agent query (sentiment-specific)
single_query = "What is the recent news sentiment for CIMB?"

print(f"Query: {single_query}")
print("-"*40)

# Classify first to show routing decision
classification = orchestrator.classify_query(single_query)
print(f"Classification:")
print(f"  Needs Debate: {classification['needs_debate']}")
print(f"  Agent: {classification['agent_type']}")
print(f"  Company: {classification.get('company')}")

# Route the query
result = orchestrator.route_query(single_query)

print(f"\nRoute Type: {result['route_type']}")
print(f"Agent Used: {result['agent_used']}")
print(f"\nResponse Preview:")
print(result['response'][:800] + "...")

In [ ]:
# Example: Valuation-specific query
valuation_query = "What is the volatility and Sharpe ratio for Maybank?"

print(f"Query: {valuation_query}")
print("-"*40)

result = orchestrator.route_query(valuation_query)

print(f"Route Type: {result['route_type']}")
print(f"Agent Used: {result['agent_used']}")
print(f"\nResponse Preview:")
print(result['response'][:800] + "...")

## Conservative Investor Analysis

In [7]:
# Conservative investor query - explicit risk tolerance in language
conservative_query = "I'm a retiree looking for a safe, stable dividend stock. Should I invest in Malayan Banking for capital preservation?"

print(f"Query: {conservative_query}")
print(f"\n{'='*60}")
print("Running analysis for CONSERVATIVE investor...")
print(f"{'='*60}\n")

# Route query - will infer conservative risk tolerance
result = orchestrator.route_query(conservative_query)

# Debug: Show classification details
print("Classification Details:")
classification = result.get('classification', {})
print(f"  Needs Debate: {classification.get('needs_debate')}")
print(f"  Agent Type: {classification.get('agent_type')}")
print(f"  Company: {classification.get('company')}")
print(f"  Reasoning: {classification.get('reasoning')}")

print(f"\nRoute Type: {result['route_type']}")
print(f"Inferred Risk Tolerance: {result['risk_tolerance'].upper()}")
print(f"Recommendation: {result.get('recommendation', 'N/A')}")

if result['route_type'] == 'debate':
    print(f"Confidence: {result['confidence']:.0%}")
    print(f"Consensus: {result['consensus']:.0%}")
elif result['route_type'] == 'single_agent':
    print(f"Agent Used: {result['agent_used']}")
    print(f"\nResponse Preview:\n{result['response'][:500]}...")
elif result['route_type'] == 'error':
    print(f"Error: {result['response']}")

Query: I'm a retiree looking for a safe, stable dividend stock. Should I invest in Malayan Banking for capital preservation?

Running analysis for CONSERVATIVE investor...

[Round 1] FUNDAMENTAL: HOLD (80% confidence)
[Round 1] FUNDAMENTAL: HOLD (80% confidence)
[values] {'messages': [HumanMessage(content="Analyze MALAYAN BANKING BERHAD (1155.KL) from a MARKET SENTIMENT perspective.\n\n**INVESTOR PROFILE**: The investor is CONSERVATIVE - they prioritize capital preservation, stable income, and low volatility over high returns. They prefer established companies with consistent dividends and minimal downside risk.\n\nYou are the Sentiment Analysis Agent. Your role is to evaluate market perception, news sentiment, and investor mood based on recent news articles and FinBERT sentiment scores.\n\nPerform your analysis and provide:\n\n1. **Key Findings**: What does recent news sentiment reveal?\n   - Overall sentiment distribution (positive/negative/neutral percentages)\n   - Key themes in re

In [ ]:
# Aggressive investor query
aggressive_query = "I want maximum growth and I'm willing to take high risks. Should I buy CIMB for aggressive returns?"

print(f"Query: {aggressive_query}")
print(f"\n{'='*60}")
print("Running analysis for AGGRESSIVE investor...")
print(f"{'='*60}\n")

# Route query - will infer aggressive risk tolerance
result = orchestrator.route_query(aggressive_query)

print(f"\nInferred Risk Tolerance: {result['risk_tolerance'].upper()}")
print(f"Recommendation: {result.get('recommendation', 'N/A')}")
if result['route_type'] == 'debate':
    print(f"Confidence: {result['confidence']:.0%}")
    print(f"Consensus: {result['consensus']:.0%}")

## View Debate Transcript

In [8]:
# Get debate transcript from the most recent debate
transcript = orchestrator.get_debate_transcript()
print(transcript)

DEBATE TRANSCRIPT: MALAYAN BANKING BERHAD (1155.KL)
Started: 2025-12-14 21:53:49


--- ROUND 1 ---

[FUNDAMENTAL] (HOLD, 80% confidence)
----------------------------------------
Okay, I will analyze MALAYAN BANKING BERHAD (1155.KL) based on the provided financial data and the investor's conservative risk profile.

**1. Key Findings:**

*   **Revenue and Profit Trends:**
    *   Total Revenue has shown consistent growth from 23.82B MYR in 2021 to 28.03B MYR in 2024.
    *   Net Income has also increased from 8.10B MYR in 2021 to 10.09B MYR in 2024.
    *   This indicates a positive trend in both revenue generation and profitability.
*   **Financial Ratios:**
    *   **P/E Ratio:** The P/E ratio is around 11.61, which suggests the stock might be reasonably valued compared to its earnings.
    *   **ROE:** The Return on Equity (ROE) is approximately 11.32%, indicating good profitability relative to shareholders' equity.
    *   **Debt-to-Equity:** The debt-to-equity ratio cannot be direct